In [1]:
# @zhangjoy
# @主题挖掘
import csv
import time
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import tushare as ts
import jieba
import re
from sklearn.feature_extraction.text import CountVectorizer # 词频统计
from sklearn.decomposition import LatentDirichletAllocation # LDA
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer # tf-idf统计
from sklearn.cluster import KMeans # k均值聚类

In [3]:
# 评论文本：comment
filepath = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/20180201-20180301.csv"   # 评论内容的存储路径
with open(filepath, 'r') as f:
    reader = csv.reader(f)
    comment = [row[7] for row in reader]
comment.pop(0) # '评论内容'
print(len(comment)) # 6710

6709


In [4]:
comment[6708]

'来源：凤凰网 编辑：东方财富网北京时间2月1日03:00，美联储主席耶伦将最后一次主持美联储利率会议。接下来，鲍威尔将执掌美联储，成为新一任主席。2014年，耶伦接任美联储主席，成为首位女性掌门人。当时，美联储在前任主席伯南克的领导下实行量化宽松的货币政策。量化宽松主要是指美联储通过从金融机构手中购买债券，增加基础货币供给。量化宽松的货币政策为美国经济注入了数十亿美元的资金，以维持金融危机之后美国经济的正常运行。下个月，耶伦将卸任美联储主席一职。她的功绩就是开启了停止量化宽松这个台“机器”的漫长征程，并在2015年7月宣布了七年来的首次加息。美国的利率在金融危机期间一直保持平稳，但现在又开始上调。由于美国股市处于历史高位，美国经济增长继续走强，失业率正处于2000年以来的最低水平，因此，鲍威尔接下来的任务将十分艰巨。虽然今年晚些时候还会有进一步加息，但预计本月不会加息。荷兰国际集团资深经济学家詹姆斯表示：“耶伦强有力的领导力和强硬的决策能力，让美国经济表现强劲。鉴于美国经济取得的这些成就，下一任主席鲍威尔将面临非常严格的门槛标准。”以下是耶伦取得的几项成就：充分就业在卸任美联储主席时，耶伦已经完成了美国国会设置的关键经济目标，即最大化美国就业率。在耶伦的领导下，美国失业率在过去4年里下降了三分之一以上。去年12月，美国失业率水平从耶伦上任伊始的6.7%下降到4.1%，创2000年以来的最低水平。由于美联储的长期失业率预估水平为4.6%，因此加息的时机已经出现。但是，鲍威尔需谨慎行事，以使得特朗普总统满意。据悉，特朗普胜选的重要一点是他承诺将重点改善美国的就业问题。通胀稳定且通胀速度缓慢耶伦也完成了美国国会设定的第二个目标：将通货膨胀水平保持在稳定且缓慢波动的范围内，美国物价增长水平接近2%的目标水平。尽管2015年，美国消费者价格指数受通货紧缩影响接近她初任美联储主席时的水平，但消费者价格近年来一直在稳步增长，没有出现失控的局面。去年12月，美国通胀率为2.1%。但是，耶伦也必须解决一个难题，这个难题鲍威尔接手之后也会面对：尽管美国失业率处于2000年以来的最低水平，但奇怪的是美国工人的工资仍然没有出现增长。因此，美联储需要确保在进一步加息之前，工人的工资将会增长。去年，耶伦开始缩减美联储4.5 万亿美元(合3.2万亿英镑)的资产负债表，同时，投资者也对她逐渐退出

In [26]:
# comment可视化分析
l_lst = [len(c) for c in comment]

In [27]:
max(l_lst)

18037

In [28]:
min(l_lst)

17

In [30]:
sum(l_lst) / len(l_lst)

1521.9317334923237

In [44]:
t_lst = [466,511,348,195,689,584,451,355,315,128,123,195,202,178,52,50,51,32,48,43,56,303,302,131,67,272,299,263]

In [45]:
sum(t_lst)

6709

In [5]:
# 得到所有股票的代码和中文名字
stock_name = ts.get_stock_basics()['name'] # 股票名称
stock_code = stock_name.index.tolist() # 股票代码
# 获得所有的概念列表、行业列表、地域列表、公司名称等
concept = ts.get_concept_classified()['c_name'].drop_duplicates().values.tolist() # 概念列表 163
industry = ts.get_industry_classified('sw')['c_name'].drop_duplicates().values.tolist() # 申万行业列表 129 > 110
place = ts.get_area_classified()['area'].drop_duplicates().values.tolist() # 获取地域列表 32 省份

broker = ts.broker_tops()["broker"] # 获取营业部信息：1062
pattern = re.compile(u'.*有限.*?公司') # 最小匹配[*?]到'有限 公司'
company = [" ".join(pattern.findall(s)) for s in broker] # list
company = pd.Series(company).drop_duplicates().values.tolist() # 证券公司：94

[Getting data:]###########################

In [6]:
# 将上述信息作为新词录入词典
for d in stock_name:
    jieba.add_word(d)
for d in stock_code:
    jieba.add_word(d)
for d in concept:
    jieba.add_word(d)
for d in industry:
    jieba.add_word(d)
for d in place:
    jieba.add_word(d)    
for d in company:
    jieba.add_word(d)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/xv/52pzd4c57hb49kf41gxp713w0000gn/T/jieba.cache
Loading model cost 0.927 seconds.
Prefix dict has been built succesfully.


In [7]:
# 启用自定义词典
jieba.load_userdict("/Users/zhangjoy/Documents/vscode/py_project/lda/data/userdict.txt")

In [8]:
# 停用词列表
stop =[line.strip() for line in open('/Users/zhangjoy/Documents/vscode/py_project/lda/data/stopwords.txt').readlines()]

In [9]:
# 匹配中文
pattern = re.compile(u'[\u4e00-\u9fa5]{2,20}')

In [10]:
text = [] # 清洗后文本
t1 = time.time()
for sentence in comment:
    chinese = pattern.findall(sentence) # 匹配中文：先分句
    seg = [jieba.cut(c) for c in chinese] # 双重列表：句上分词
    c = []
    for ss in seg:
        for s in ss:
            if s not in stop: # 过滤停用词
                c.append(s)
    text.append(" ".join(c))
t2 = time.time()
print("preprocessing time using %.5f s" %(t2-t1)) # 预处理时间 111.45185 s

preprocessing time using 117.78608 s


In [11]:
text[6708]

'北京时间 美联储 主席 耶伦 最后一次 主持 美联储 利率 鲍威尔 执掌 美联储 成为 新一任 主席 耶伦 接任 美联储 主席 成为 首位 女性 掌门人 当时 美联储 前任 主席 伯南克 领导 实行 量化 宽 松 货币政策 量化 宽松 主要 指 美联储 金融机构 手中 购买 债券 增加 基础 货币 供给 量化 宽松 货币政策 美国 经济 注入 数十亿 美元 资金 维持 金融危机 美国 经济 正常 运行 下个月 耶伦 卸任 美联储 主席 一职 功绩 开启 停止 量化 宽松 台 机器 漫长 征程 宣布 首次 加息 美国 利率 金融危机 期间 保持 平稳 上调 美国股市 处于 历史 高位 美国 经济 增长 走强 失业率 正 处于 最低水平 鲍威尔 任务 十分艰巨 晚些时候 还会 进一步 加息 预计 本月 加息 荷兰 国际 集团 资深 经济学家 詹姆斯 耶伦 强有力 领导力 强硬 决策 能力 美国 经济 表现 强劲 美国 经济 取得 成就 一任 主席 鲍威尔 面临 严格 门槛 标准 耶伦 取得 几项 成就 充分就业 卸任 美联储 主席 时 耶伦 完成 美国国会 设置 关键 经济 目标 最大化 美国 就业率 耶伦 领导 美国 失业率 过去 下降 三分之一 美国 失业率 水平 从耶伦 上任 伊始 下降 最低水平 美联储 长期 失业率 预估 水平 加息 时机 出现 鲍威尔 需谨慎 行事 特朗普 总统 满意 特朗普 胜选 重要 一点 承诺 重点 改善 美 国 就业 通胀 稳定 通胀 速度 缓慢 耶伦 完成 美国 国 设定 第二个 目标 通货膨胀 水平 保持 稳定 缓慢 波动 范围 美国 物价 增长 水平 接近 目标 水平 美国 消费者 价格指数 受 通货紧缩 影响 接近 初 美联储 主席 时 水平 消费者 价格 稳步增长 出现 失控 局面 美国 通胀率 耶伦 解决 难题 难题 鲍威尔 接手 面对 美国 失业率 处于 最低水平 奇怪的是 美国 工人 工资 出现 增长 美联储 需要 确保 进一步 加息 之前 工人 工资 将会 增长 耶伦 缩减 美联储 万亿美元 万 亿英镑 资产负债表 投资者 逐渐 退出 经济 刺激 计划 做法 予 赞赏 相比之下 一任 美联储 主席 伯南克 信贷 紧缩 时期 推出 量化 宽松 货币政策 陷入 混乱 当时 伯南克 首次 直播 发布 展望 耶伦 领导 美

In [19]:
# 使第6708条评论完美分词
jieba.load_userdict("/Users/zhangjoy/Documents/vscode/py_project/lda/data/userdict.txt")
stop =[line.strip() for line in open('/Users/zhangjoy/Documents/vscode/py_project/lda/data/stopwords.txt').readlines()]
text_6708 = "" # 清洗后文本
comment_6708 = comment[6708]
chinese = pattern.findall(comment_6708) # 匹配中文：先分句
seg = [jieba.cut(c) for c in chinese] # 双重列表：句上分词
c = []
for ss in seg:
    for s in ss:
        if s not in stop: # 过滤停用词
            c.append(s)
text_6708 = " ".join(c)
print(text_6708)

北京时间 美联储 主席 耶伦 最后一次 主持 美联储 利率 鲍威尔 执掌 美联储 成为 新一任 主席 耶伦 接任 美联储 主席 成为 首位 女性 掌门人 当时 美联储 前任 主席 伯南克 领导 实行 量化 宽 松 货币政策 量化 宽松 主要 指 美联储 金融机构 手中 购买 债券 增加 基础 货币 供给 量化 宽松 货币政策 美国 经济 注入 数十亿 美元 资金 维持 金融危机 美国 经济 正常 运行 下个月 耶伦 卸任 美联储 主席 一职 功绩 开启 停止 量化 宽松 台 机器 漫长 征程 宣布 首次 加息 美国 利率 金融危机 期间 保持 平稳 上调 美国股市 处于 历史 高位 美国 经济 增长 走强 失业率 正 处于 最低水平 鲍威尔 任务 十分艰巨 晚些时候 还会 进一步 加息 预计 本月 加息 荷兰 国际 集团 资深 经济学家 詹姆斯 耶伦 强有力 领导力 强硬 决策 能力 美国 经济 表现 强劲 美国 经济 取得 成就 一任 主席 鲍威尔 面临 严格 门槛 标准 耶伦 取得 几项 成就 充分就业 卸任 美联储 主席 时 耶伦 完成 美国国会 设置 关键 经济 目标 最大化 美国 就业率 耶伦 领导 美国 失业率 过去 下降 三分之一 美国 失业率 水平 从耶伦 上任 伊始 下降 最低水平 美联储 长期 失业率 预估 水平 加息 时机 出现 鲍威尔 需谨慎 行事 特朗普 总统 满意 特朗普 胜选 重要 一点 承诺 重点 改善 美 国 就业 通胀 稳定 通胀 速度 缓慢 耶伦 完成 美国 国 设定 第二个 目标 通货膨胀 水平 保持 稳定 缓慢 波动 范围 美国 物价 增长 水平 接近 目标 水平 美国 消费者 价格指数 受 通货紧缩 影响 接近 初 美联储 主席 时 水平 消费者 价格 稳步增长 出现 失控 局面 美国 通胀率 耶伦 解决 难题 难题 鲍威尔 接手 面对 美国 失业率 处于 最低水平 奇怪的是 美国 工人 工资 出现 增长 美联储 需要 确保 进一步 加息 之前 工人 工资 将会 增长 耶伦 缩减 美联储 万亿美元 万 亿英镑 资产负债表 投资者 逐渐 退出 经济 刺激 计划 做法 予 赞赏 相比之下 一任 美联储 主席 伯南克 信贷 紧缩 时期 推出 量化 宽松 货币政策 陷入 混乱 当时 伯南克 首次 直播 发布 展望 耶伦 领导 美联

In [12]:
corpus = text # 清洗后文本作为语料

In [13]:
# LDA
# 特征处理: 词频做文档特征
# 构建 Document-word matrix
count_vectorizer = CountVectorizer(stop_words=stop) # 停用词, 特征选择：min_df = 6
count = count_vectorizer.fit_transform(corpus) # 文档-词频矩阵：<class 'scipy.sparse.csr.csr_matrix'>
word_names = count_vectorizer.get_feature_names() # wordlist 长度：84888

In [68]:
countpie_vectorizer = CountVectorizer(min_df=50,stop_words=stop) # 过滤词出现的文档数 < 50 的词 ：特征提取
countpie = countpie_vectorizer.fit_transform(corpus) # 文档-词频矩阵：<class 'scipy.sparse.csr.csr_matrix'>
print(countpie.shape)

(6709, 4385)


In [20]:
c_lst = [count.sum(axis=0)[0,i] for i in range(len(word_names))] # 每个词的词频列表 耗时：5m

In [24]:
less_6 = sum([Counter(c_lst)[i] for i in range(6)]) # 词频 < 6 的个数

59454

In [43]:
# 特征选择：基于文档频率，返回文档频率排名前 K 的词语
def word_count_select(df,K):
    df_mean = df.mean() # 求每列的均值/和等
    # 基于 文档频率排序 选择特征，一定返回K个词语
    WordTF_series = df_mean.sort_values(ascending=False) # 降序排序
    WordTF_topK = WordTF_series[:K] # 选择前 K 的特征
    word_feature = WordTF_topK.index # word_name [pandas.core.indexes.base.Index]
    feature_list = list(WordTF_topK.index) # list(word_name)
    return word_feature,feature_list
# 词频小于 6 的个数统计
def count_less_n(lst,N):
    lessN = sum([Counter(lst)[i] for i in range(N)]) # 词频 < N 的个数
    return lessN

In [ ]:
# 把文档-词语矩阵转为数据框格式
count_ndarray = count.toarray()
count_df = DataFrame(count_ndarray, colunms=word_names)
k = len(word_names) - count_less_n(c_lst,6) # 词频 > 6 的个数

wordstay_names,wordstay_names_list = word_count_select(count_df,k)
count_df2 = count_df[wordstay_names]
countstay = count_df2.as_matrix(columns=wordstay_names_list) # matrix

In [49]:
# count # 6709x84884 sparse matrix numpy.int64
# 去掉低频词：在主题模型中，数据预处理需要去掉低频词，大多数阈值设定为5或6,此处设定为6
# matrix是numpy中的特殊类型，作为array的子类出现，所以继承了array的所有特性并且有自己的特殊的地方，专门用来处理线性代数操作

numpy.matrixlib.defmatrix.matrix

In [81]:
# 词频数分布情况
f = open('/Users/zhangjoy/Documents/vscode/py_project/lda/data/count_1.txt','w')
f.write(str([Counter(c_lst).most_common(1178)[i][0] for i in range(1178)]))
f.close()
f = open('/Users/zhangjoy/Documents/vscode/py_project/lda/data/count_2.txt','w')
f.write(str([Counter(c_lst).most_common(1178)[i][1] for i in range(1178)]))
f.close()

In [50]:
# 主题挖掘建模：LDA算法
# 训练模型
k = 50 # 选择主题数
lda_50 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_50.fit(countstay)
# 6709文档，单词数：25434，分50个主题 耗时：< 10m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=50, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [51]:
# 训练模型
k = 30 # 选择主题数
lda_30 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_30.fit(countstay)
# 6709文档，单词数：25434，分30个主题 耗时：12:54-13:01 7m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=30, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [52]:
# 训练模型
k = 10 # 选择主题数
lda_10 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_10.fit(countstay)
# 6709文档，单词数：25434，分10个主题 耗时：13:02-13:08 6m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [111]:
# 结果展示
# Topic Words 结果
# topicword_matrix = lda.components_ # numpy.ndarray
# print(topicword_matrix[0,1]) # topicword_matrix[i,j]: word j was assigned to topic i
# print(topicword_matrix.shape)
# topicword_matrix / topicword_matrix.sum(axis=1)[:, np.newaxis] # normalization

# Topic（Top）Words结果
def print_top_words(model, feature_names, n_top_words):
    print("# 打印每个主题下权重较高的词")
    topicword_matrix = model.components_ # Topic Words 分布结果 numpy.ndarray
    for topic_idx, topic in enumerate(topicword_matrix):
        print("Topic # %d: " % topic_idx)
        print(''.join([feature_names[i] + ' ' + str(round(topic[i], 2)) + '| ' for i in topic.argsort()[:-n_top_words - 1:-1]]))
        # print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

def print_top_words_name(model, feature_names, n_top_words):
    print("# 打印每个主题下权重较高的词")
    topicword_matrix = model.components_
    for topic_idx, topic in enumerate(topicword_matrix):
#         print("Topic # %d:" % topic_idx)
        print("Topic %d:" % topic_idx, " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [55]:
# Doc Topics 结果
# 通过fit_transform函数,可以得到文档主题分布,Doc_Topic distribution matrix
# doctopic_matrix = lda.fit_transform(dw_countvector) # numpy.ndarray
# print(doctopic_matrix)
# print(doctopic_matrix.shape)

# Doc（Top）Topic结果
def print_top_topics(model, feature_names, n_top_topics):
#     fo = open("/Users/zhangjoy/Documents/vscode/py_project/lda/data/doc_top_topics.txt", "rw+")
    print("# 打印每个文档下权重较高的主题")
    doctopic_matrix = model.fit_transform(countstay)
    for doc_idx, doc in enumerate(doctopic_matrix):
        print("Doc # %d:" % doc_idx)
        print(''.join([str(feature_names[i]) + ' ' + str(round(doc[i], 2)) + '| ' for i in doc.argsort()[:-n_top_topics - 1:-1]]))

In [56]:
print_top_words_name(lda_50, wordstay_names_list, 10)

# 打印每个主题下权重较高的词
万科 英国 经济 中国 美国 主席 特朗普 董事 薪酬 利润
建设 规划 雄安新区 会议 海南 合作 北京 发展 习近平 旅游
项目 企业 政府 环保 税收 快递 示范 信用 记者 环境
进口 阿胶 中国 药品 股份 美国 出口 发射 国家 火箭
期货 原油 石油 原油期货 天然气 合约 价格 国内 中国 国际
汽车 新能源 电池 补贴 万辆 企业 回收 材料 动力电池 全球
罚款 处罚 陕西 销售 计算机 行政处罚 河南 化工 机械设备 生产
腾讯 阿里 阿里巴巴 零售 集团 京东 合作 商业 苏宁 新零售
消费 品牌 消费者 销售 产品 电商 渠道 春节 网点 用户
券商 旅游 证券 旅客 航空 资管 规模 业务 机场 铁路
银行 贷款 房地产 城市 租赁 住房 理财 资产 规模 资金
反弹 创业板 下跌 行情 指数 出现 个股 板块 调整 大盘
行业 板块 业绩 估值 有望 关注 预期 龙头 提升 投资
人民币 电影 票房 春节 汇率 升值 影视 影片 行动 乖离
估值 美股 港股 购买 调整 基金 资金 投资者 指数 投资
白酒 信托 茅台 价格 私募 检查 管理 记者 证监局 行业
平安 服务 医疗 健康 医院 医生 支付 自动 医保 驾驶
区块 货币 比特币 富士康 平台 交易 监管 虚拟 相关 数字
基金 机构 调研 持有 产品 经理 投资 净值 规模 记者
交易 证券 成交 账户 卖出 大宗 机构 万股 回购 金额
增长 同比 净利润 业绩 实现 预计 收入 下降 显示 数据
股东大会 年度 临时 召开 公告 停牌 基金 管理 发行 披露
农村 乡村 建设 发展 振兴 农业 战略 改革 中央 文件
保险 保费 财险 保险公司 保监会 增速 人寿 产品 业务 车险
增持 分红 股东 股份 上市公司 公告 现金 万股 计划 控股
经济 中国 流动性 央行 金融 利率 杠杆 风险 货币 增速
手机 全球 视频 中国 网络 芯片 财富 小米 厂商 爱奇艺
煤炭 价格 库存 上涨 春节 需求 全国 企业 供应 指数
质押 股票 股东 信托 平仓 风险 股权 计划 预增 业务
个股 资金 净流入 主力 买入 机构 数据 证券 金额 流出
改革 证券 买入 企业 注册 混改 目标 天风 混合 所有制
特斯拉 数据 北斗 单车 驾驶 

In [57]:
print_top_words_name(lda_30, wordstay_names_list, 10)

# 打印每个主题下权重较高的词
记者 视频 时间 看到 需要 游戏 发现 这种 知道 生活
中国 经济 发展 企业 投资 我国 增长 美国 国家 消费
基金 注册 万科 发行 改革 巴菲特 股票 董事 授权 董事会
技术 中国 区块 互联网 货币 比特币 科技 数据 人工智能 智能
信托 项目 投资 业务 计划 产品 管理 杠杆 私募 资产
旅游 海南 两会 铁路 航空 旅客 快递 春节 游客 北京
汽车 新能源 企业 国内 销量 全球 行业 增长 产能 产品
产品 业务 服务 研发 富士康 股份 工业 技术 预增 项目
美股 经济 风险 调整 股市 影响 流动性 下跌 全球 估值
腾讯 万达 品牌 合作 投资 集团 阿里 零售 业务 战略
集团 企业 中国 项目 核电 重组 能源 投资 国家 完成
乐视网 乐视 贾跃亭 网络 海航 充电 资金 孙宏斌 金立 测试
行业 业绩 板块 估值 增长 基金 关注 有望 龙头 投资
农村 乡村 农业 振兴 建设 改革 发展 文件 中央 战略
上市 企业 交易 港股 发行 新股 香港 投资者 申购 市值
个股 买入 机构 证券 卖出 涨幅 金额 资金 数据 增持
券商 融资 证券 业务 规模 行业 项目 电视 余额 光伏
股东 股份 质押 公告 增持 股权 停牌 控股 股票 万股
银行 金融 监管 贷款 风险 资产 业务 资金 融资 金融机构
发展 建设 创新 国家 企业 推动 产业 改革 经济 服务
股票 股价 投资者 基金 投资 出现 下跌 跌停 市值 资金
资金 临时 召开 年度 股东大会 净流入 流出 主力 连续 北上
增长 人民币 同比 增速 下降 数据 环比 汇率 保险 升值
业绩 净利润 增长 同比 上市公司 分红 预计 实现 亏损 年度
城市 房地产 租赁 住房 房企 销售 北京 地产 房价 调控
美元 美国 指数 预期 美联储 经济 数据 黄金 加息 增长
春节 电影 票房 阿胶 影视 影片 行动 内容 军工 捉妖
监管 相关 交易 规定 要求 信息 存在 进行 行为 证监会
板块 反弹 行情 指数 创业板 个股 上涨 下跌 调整 大盘
价格 企业 库存 白酒 需求 产能 煤炭 上涨 国内 生产


In [58]:
print_top_words_name(lda_10, wordstay_names_list, 10)

# 打印每个主题下权重较高的词
增长 业绩 个股 同比 行业 净利润 预计 买入 证券 数据
银行 监管 业务 金融 风险 贷款 规模 融资 资产 投资
中国 企业 经济 增长 投资 消费 发展 增速 行业 需要
发展 建设 企业 改革 经济 国家 中国 项目 乡村 农村
美国 美元 经济 预期 指数 人民币 美联储 美股 股市 全球
股份 股东 公告 质押 增持 股票 交易 发行 停牌 控股
估值 下跌 指数 调整 板块 资金 出现 投资者 行情 风险
行业 企业 技术 业务 互联网 发展 服务 产品 中国 平台
基金 上市公司 分红 投资 股东 投资者 乐视网 亏损 业绩 现金
汽车 中国 新能源 春节 万达 增长 国内 销量 同比 电影


In [108]:
# 训练模型
k = 20 # 选择主题数
lda_20 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_20.fit(countstay)
# 6709文档，单词数：25434，分20个主题 耗时：17:28-17:35 7m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=20, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [112]:
print_top_words_name(lda_20, wordstay_names_list, 15)

# 打印每个主题下权重较高的词
Topic 0: 记者 北京 旅客 上海 航空 海南 股份 快递 铁路 员工 机场 全国 航空公司 红包 车辆
Topic 1: 交易 港股 期货 申购 香港 上海 账户 原油期货 投资者 证券 国际 发行人 上市 参与 大宗
Topic 2: 基金 银行 券商 融资 规模 业务 投资 资金 资产 贷款 证券 数据 机构 余额 发行
Topic 3: 个股 买入 资金 机构 证券 涨幅 卖出 净流入 金额 主力 数据 成交 涨停 跌幅 板块
Topic 4: 质押 股票 股东 停牌 乐视网 股价 上市公司 公告 信托 平仓 跌停 股权 亏损 披露 计划
Topic 5: 增长 业绩 同比 净利润 预计 行业 实现 增速 盈利 收入 提升 上市公司 板块 预告 年报
Topic 6: 投资 经济 中国 股市 美国 资产 出现 风险 价值 投资者 过去 全球 股票 未来 这种
Topic 7: 监管 企业 风险 上市 资本 上市公司 发行 投资者 要求 证监会 交易 改革 制度 进行 相关
Topic 8: 企业 行业 中国 价格 产品 国内 生产 新能源 需求 汽车 产能 产业 全球 发展 增长
Topic 9: 相关 银行 银监会 银行业 行为 违规 处罚 调查 记者 罚款 案件 规定 贷款 存在 金融机构
Topic 10: 城市 房地产 租赁 住房 销售 房企 地产 企业 房价 调控 土地 数据 楼市 北京 发展
Topic 11: 美元 人民币 美国 货币 中国 黄金 英国 经济 全球 比特币 汇率 升值 价格 交易 央行
Topic 12: 发展 建设 经济 中国 改革 国家 农村 乡村 创新 企业 推动 战略 农业 产业 振兴
Topic 13: 板块 估值 指数 下跌 调整 行情 反弹 出现 资金 上涨 个股 创业板 投资者 业绩 风险
Topic 14: 互联网 中国 腾讯 技术 万达 行业 企业 发展 成为 业务 平台 服务 用户 合作 领域
Topic 15: 汽车 万科 乐视 销量 特斯拉 万辆 贾跃亭 电视 董事 吉利 董事会 利润 计划 车型 薪酬
Topic 16: 美国 指数 分红 预期 美联储 美股 加息 临时 召开 年度 股市 股东大会 通胀 收益率 经济
Topic 17: 保险 区块 金融 业务 平

In [123]:
# 训练模型
k = 20 # 选择主题数
lda_20 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_20.fit(dw_countvector)
# 6709文档，单词数：84884，分20个主题 耗时：2-3m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=20, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [134]:
print_top_words_name(lda_20, word_names, 15)

# 打印每个主题下权重较高的词
Topic # 0:
上市 券商 股东大会 临时 召开 年度 企业 发行 新股 证券 万科 申购 项目 管理 数量
Topic # 1:
增持 股份 公告 股东 万股 回购 控股 计划 预增 票房 电影 项目 股本 投资 低于
Topic # 2:
巴菲特 投资 觉得 这种 需要 看到 财富 能够 价值 应该 养老 知道 时间 比较 这是
Topic # 3:
银行 人民币 融资 业务 资产 信托 贷款 资金 规模 金融 杠杆 监管 投资 理财 资管
Topic # 4:
中国 产品 技术 汽车 企业 行业 发展 新能源 全球 产业 增长 国内 业务 领域 服务
Topic # 5:
业绩 净利润 增长 同比 上市公司 分红 预计 实现 披露 亏损 年报 收入 股东 预告 年度
Topic # 6:
货币 监管 央行 比特币 交易 金融 流动性 银行业 风险 金融机构 银行 银监会 机构 存款 中国
Topic # 7:
集团 企业 光伏 两会 混改 投资 混合 项目 重组 所有制 海南 员工 国资委 记者 央企
Topic # 8:
乐视网 交易 港股 融资 卖出 乐视 贾跃亭 资金 余额 证券 万股 基金 投资者 是否 发行人
Topic # 9:
基金 城市 房地产 住房 租赁 销售 房企 地产 数据 规模 企业 房价 记者 北京 调控
Topic # 10:
增长 行业 增速 价格 同比 需求 有望 提升 预计 持续 预期 产能 龙头 板块 业绩
Topic # 11:
经济 美股 投资者 下跌 估值 风险 出现 调整 指数 股市 投资 影响 预期 中国 股票
Topic # 12:
发展 建设 经济 企业 国家 中国 改革 农村 创新 乡村 产业 战略 推动 农业 服务
Topic # 13:
白酒 特斯拉 茅台 发射 火箭 雄安新区 价格 驾驶 记者 自动 卫星 北斗 高端 马斯克 春节
Topic # 14:
个股 板块 资金 买入 机构 涨幅 证券 创业板 上涨 数据 反弹 行情 跌幅 行业 两市
Topic # 15:
腾讯 互联网 投资 平台 万达 企业 中国 集团 阿里 行业 用户 区块 成为 零售 业务
Topic # 16:
美国 中国 英国 全球 特朗普 收购 经济 报道 进口 贸易 日本 香港 政府 国家

In [62]:
def top_topics(model,topN):
    doctopic_matrix = model.fit_transform(countstay)
    doctopic = doctopic_matrix.argmax(axis=1).tolist() # 取最大值时的索引
    print("# 打印前 %d 热门主题的序号" % topN)
    print([Counter(doctopic).most_common(topN)[i][0] for i in range(topN)]) # 找出列表元素最多的N个,结果是list类型

In [63]:
top_topics(lda_50,10)
# 6709个文档，lda_50，前10个主题，耗时： 8 m 

# 打印前 10 热门主题的序号
[41, 6, 31, 23, 29, 14, 30, 20, 44, 38]


In [142]:
# 效果等同上述函数
doctopic_matrix = lda_20.fit_transform(countstay)
doctopic = doctopic_matrix.argmax(axis=1).tolist() # 取最大值时的索引
print("# 打印前 %d 热门主题的序号" % topN)
c = Counter(doctopic)
c.most_common(10) # 找出列表元素最多的N个,结果是list类型

In [76]:
# 主题挖掘建模：文本聚类算法
# 特征处理: TF-IDF做文档特征
tfidf_vectorizer = TfidfVectorizer(stop_words=stop,max_df=0.9,min_df=10) # 特征提取
tfidf = tfidf_vectorizer.fit_transform(corpus) # matrix 
word_names = tfidf_vectorizer.get_feature_names() # list
# tfidf矩阵:元素a[i][j]表示j词在i文档中的tf-idf权重
# weight = tfidf.toarray()  # matrix -> ndarray
print(len(word_names))

14839


In [75]:
# 定义余弦相似度
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf)

In [78]:
# keans
# 训练模型
k = 20
km_20 = KMeans(n_clusters=k)
%time km_20.fit(tfidf)
clusters = km_20.labels_.tolist() # 每个文档所属类别

CPU times: user 3min 36s, sys: 2.12 s, total: 3min 38s
Wall time: 3min 41s


In [109]:
print("Top terms per cluster:")
print()
# 按离质心的距离排列聚类中心，由近到远
order_centroids = km_20.cluster_centers_.argsort()[:, ::-1] 

for i in range(k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :15]: # 每个聚类选 15 个词
        print(' %s' % word_names[ind], end=' ')
    print() # 空行

Top terms per cluster:

Cluster 0: 人民币  美元  汇率  升值  中间价  在岸  美元汇率  货币  基点  中国  贬值  指数  官方  交易日  收盘价 
Cluster 1: 增持  股份  公告  股东  回购  万股  控股  股本  计划  公司股票  减持  上市公司  晚间  员工  低于 
Cluster 2: 卖出  买入  机构  金额  个股  龙虎榜  营业部  融资  当天  资金  数据  交易  万股  席位  证券 
Cluster 3: 美股  股市  下跌  指数  美国  暴跌  经济  估值  大跌  调整  跌幅  全球  影响  风险  预期 
Cluster 4: 企业  发展  中国  经济  改革  建设  产业  金融  创新  项目  资本  国家  投资  行业  服务 
Cluster 5: 板块  反弹  行情  个股  创业板  指数  估值  业绩  大盘  资金  调整  下跌  春节  上涨  震荡 
Cluster 6: 基金  比特币  货币  开户  港股  投资  交易  购买  股票  虚拟  乐视网  投资者  持有  资金  指数 
Cluster 7: 分红  现金  上市公司  股息  说明会  上交所  投资者  回报  监管  铁公鸡  具备  东安动力  分配利润  利润分配  联美控股 
Cluster 8: 中国  股份  增长  投资  企业  集团  业务  上市  产品  公告  发行  项目  行业  销售  股东 
Cluster 9: 主力  净流入  资金  流出  个股  乖离  三日  涨跌幅  证券  净流量  最多  排名  金额  半年线  概念 
Cluster 10: 业绩  净利润  增长  同比  预计  预告  行业  年报  板块  实现  上市公司  增速  个股  收入  披露 
Cluster 11: 美联储  美国  加息  预期  收益率  经济  指数  通胀  前值  美元  股市  鲍威尔  美股  利率  国债 
Cluster 12: 万达  腾讯  阿里  零售  饿了么  新零售  步步高  京东  线下  王健林  商业  阿里巴巴  合作  集团  万达电影 
Cl

In [113]:
km_20.labels_ # numpy.ndarray

array([17,  6, 10, ...,  8, 17, 11], dtype=int32)

In [105]:
[Counter(km_20.labels_.tolist()).most_common(20)[i][0] for i in range(20)]

[8, 4, 5, 10, 17, 3, 16, 6, 11, 14, 19, 15, 2, 9, 1, 12, 18, 0, 7, 13]

In [106]:
Counter(km_20.labels_.tolist()).most_common(20)

[(8, 2564),
 (4, 808),
 (5, 518),
 (10, 454),
 (17, 268),
 (3, 237),
 (16, 211),
 (6, 207),
 (11, 195),
 (14, 176),
 (19, 173),
 (15, 151),
 (2, 141),
 (9, 141),
 (1, 128),
 (12, 85),
 (18, 75),
 (0, 62),
 (7, 59),
 (13, 56)]